<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">Colab KSO Tutorials #5: Train YOLO (Object Detection) models</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Sept 14, 2022</h5>

# 1. Set up and requirements

### Install and import Python packages

In [ ]:
try:
  import google.colab
  import os
  IN_COLAB = True
  print("Running in Colab...")

  # Clone repo
  !git clone --recurse-submodules https://github.com/ocean-data-factory-sweden/koster_yolov4.git
  !pip install -q --upgrade pip
  !pip install -q -r koster_yolov4/requirements.txt
  
  # Fix libmagic issue
  !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

  # Solution to avoid opencv and pims library issues based on https://stackoverflow.com/questions/71204741/how-to-fix-error-module-cv2-has-no-attribute-legacy-on-python-3-7-9-and-w
  #!pip uninstall -q opencv-python -y
  #!pip uninstall -q opencv-contrib-python -y
  #!pip install -q opencv-contrib-python

  # Replace upsampling script with custom version
  os.chdir("koster_yolov4/tutorials")
  !mv ../src/upsampling.py /usr/local/lib/python3.7/dist-packages/torch/nn/modules/upsampling.py

  # Enable external widgets
  from google.colab import output
  output.enable_custom_widget_manager()

  # Ensure widgets are shown properly
  !jupyter nbextension enable --user --py widgetsnbextension
  !jupyter nbextension enable --user --py jupyter_bbox_widget

  print("All packages are installed and ready to go!")
  try:
    clear_output()
    print("All packages are installed and ready to go!")
  except:
    clear_output()
    print("There have been some issues installing the packages!")
except:
  IN_COLAB = False
  import sys
  if "torch" not in sys.modules:
    !pip install -q --upgrade pip
    !pip install -q torch==1.8.0 torchvision==0.9.0
  
  # Ensure widgets are shown properly
  !jupyter nbextension enable --user --py widgetsnbextension
  !jupyter nbextension enable --user --py jupyter_bbox_widget
  print("Running locally... you're good to go!")


In [ ]:
# Set the directory of the libraries
import sys, os
sys.path.append('..')

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Import required modules
from pathlib import Path
from ipyfilechooser import FileChooser
import kso_utils.tutorials_utils as t_utils
import kso_utils.server_utils as s_utils
import kso_utils.project_utils as p_utils
import kso_utils.t3_utils as t3
import kso_utils.t4_utils as t4
import kso_utils.t5_utils as t5
import kso_utils.t6_utils as t6
import kso_utils.t8_utils as t8
from kso_utils.yolo_utils import frame_aggregation
from kso_utils.zooniverse_utils import populate_agg_annotations
import wandb

# Model-specific imports
import yolov5.train as train
import yolov5.val as val
import yolov5.detect as detect

print("Packages loaded successfully")

# 2. Train the model

🔴 <span style="color:red">&nbsp;NOTE: To be able to train your own models, you will need access to the Koster WANDB group. You may request this access by contacting jurie.germishuys@combine.se. </span>

### Choose your project

In [ ]:
project_name = t_utils.choose_project()

In [ ]:
project = p_utils.find_project(project_name=project_name.value)

### Configure data paths

In [ ]:
# Specify path containing the images and labels folders.
output_folder = t_utils.choose_folder(project.photo_folder if not project.photo_folder == 'None' else ".", "output")

🔴 <span style="color:red">&nbsp;NOTE: To be able to train your own models, your data_path must contain a yml file for data and hyperparameters. See https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data#11-create-datasetyaml  </span>

In [ ]:
# Fix important paths
data_path, hyps_path = t5.setup_paths(output_folder.selected)
project_path = str(Path(output_folder.selected, project.Project_name.lower()))

### Choose a suitable experiment name

In [ ]:
exp_name = t5.choose_experiment_name()

### Choose model to use for training

In [ ]:
weights = t5.choose_baseline_model()

### Train model with given configuration

In [ ]:
batch_size, epochs, conf_thres = t5.choose_train_params()

In [ ]:
# Feel free to play around with the img_size parameter, which should work well by default. 
train.run(entity="koster", data=data_path, hyp=hyps_path, weights='yolov5m.pt', 
          project=project_path, name=exp_name.value,
          img_size=[720, 540], batch=int(batch_size.value),
          epochs=epochs.value, workers=1, single_cls=False, cache_images=True)

# 3. Evaluate model performance

In [ ]:
# Choose model
eval_model = FileChooser(project_path)
display(eval_model)

In [ ]:
# Find trained model weights
tuned_weights = f"{Path(project_path, eval_model.selected, 'weights', 'best.pt')}"

In [ ]:
# Evaluate YOLO Model on Unseen Test data
val.run(data=data_path, weights=tuned_weights, conf_thres=conf_thres.value, imgsz=640, half=False)

# (Optional) : 4. Enhance annotations using trained model

Enhancement uses the trained model to increase the amount of annotations in the training data. This should only be done in cases where it is absolutely necessary as bad predictions lead to worse predictions when used to train the next iteration of the model. 


🔴 <span style="color:red">&nbsp;NOTE: We recommend using a relatively high confidence threshold when enhancing trained models as low confidence predictions could significantly impact the quality of your annotated data.  </span>

In [ ]:
detect.run(weights=tuned_weights, source=output_folder.selected+"/images", imgsz=[640,640], conf_thres=0.02, save_txt=True)

### Choose run to use as enhanced annotations

In [ ]:

runs = FileChooser(".")
display(runs)

In [ ]:
!mv {output_folder}"/labels" {output_folder}"/labels_org"
!mv {runs.selected}"/labels" {output_folder}"/labels"

#### Once you have moved the new labels to the original label location, you can return to Step 2 and train your model again. 

In [ ]:
#END